# RASA bot

## Instructions (HowTo)
The following notebook creates and trains a new RASA bot that can answer questions based on the FAQ spreadsheet here.  
In order to talk to this bot follow these instructions:
1. Run all cells in this notebook
2. Create a new Terminal window
3. Run command `rasa shell` and follow the onscreen instructions
4. You can now talk to the bot via the command line interface

**Example of running the bot**
![Example running](example.png)

## Install Dependencies

In [1]:
%%capture
%pip install --upgrade pip
%pip uninstall -y tzlocal apscheduler
%pip install tzlocal==1.5.1 apscheduler==3.6.3
%pip install rasa==3.2.7
%pip install rasa[all]
!rasa init --no-prompt
%pip install spacy
!spacy download -d en_core_web_lg-3.2.0
%pip install rasa[spacy]
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

## RASA configuration

### NLU Training Data
Replace nlu.yml  
nlu.yml file contains all the intents for the classification of the user's questions.  
Read more at the following links: 
- https://rasa.com/docs/rasa/training-data-format
- https://rasa.com/docs/rasa/nlu-training-data

In [2]:
nlu_yml = """
version: "3.1"
nlu:
- intent: q1
  examples: |
    - do plant based diets prevent heart disease?
    - plant based diets prevent heart disease
- intent: q2
  examples: |
     - is miso healthy?
     - miso healthy
- intent: q3
  examples: |
    - is soy healthy for breast cancer survivors?
    - soy healthy breast cancer survivors
- intent: q4
  examples: |
    - are the health benefits of nuts limited to those eating bad diets?
    - benefits nuts
- intent: q5
  examples: |
    - which spices fight inflammation?
    - spices inflammation
- intent: q6
  examples: |
    - is keto a plant based diet?
    - keto plant based diet
- intent: q7
  examples: |
    - is it safe to get pregnant on a plant based diet?
    - safe pregnant plant based diet
- intent: q8
  examples: |
    - are vegans more prone to alzheimer?
    - vegans alzheimer
- intent: q9
  examples: |
    - is plant-based diet good for type 1 diabetes treatment?
    - plant-based diet type 1 diabetes
- intent: q10
  examples: |
    - which plants are anti-inflammatory?
    - plants anti-inflammatory
- intent: q11
  examples: |
    - do plant based diets prevent heart disease?
    - plant based diets heart disease
- intent: q12
  examples: |
    - what is a good bmi?
    - good bmi
- intent: q13
  examples: |
    - are green smoothies good for you?
    - green smoothies good
- intent: q14
  examples: |
    - are there foods with negative calories?
    - foods with negative calories
- intent: q15
  examples: |
    - is vanilla almond milk healthy? 
    - vanilla almond milk healthy
- intent: q16
  examples: |
    - do i need to add salt while eating vegetarian?
    - add salt while eating vegetarian
- intent: q17
  examples: |
    - what is a plant based diet?
    - plant based diet
- intent: q18
  examples: |
    - what are the benefits of plant based diets?
    - benefits plant based diets
- intent: q19
  examples: |
    - how much protein shall we consume a day?
    - protein day
- intent: q20
  examples: |
    - what's the one supplement vegans absolutely need to take?
    - supplement vegans
- intent: q21
  examples: |
    - what is the difference between prebiotics and probiotics?
    - difference prebiotics probiotics
- intent: greet
  examples: |
    - hey
    - hello
    - hello there
    - hi
    - good morning
    - good afternoon
    - good evening
    - how far
    - hey there
    - whats up
    - hey dude
    - goodmorning
    - goodafternoon
    - goodevening
- intent: goodbye
  examples: |
    - good by
    - later
    - good night
    - good afternoon
    - bye
    - goodbye
    - have a nice day
    - see you around
    - bye bye
    - see you later
- intent: thanks
  examples: |
    - thanks
    - thank you
    - appreciated
    - gracias
- intent: choosing_item
  examples: |
    - 1
    - 2
    - 3
    - 4
    - 5
    - 6
    - 7
    - 8
    - 9
- intent: affirm
  examples: |
    - yes
    - y
    - indeed
    - of course
    - that sounds good
    - correct
- intent: deny
  examples: |
    - no
    - n
    - never
    - I don't think so
    - don't like that
    - no way
    - not really
- intent: bot_challenge
  examples: |
    - are you a bot?
    - are you a human?
    - who is this?
    - what are you?
    - what's your name?
    - am I talking to a bot?
    - am I talking to a human?
- intent: more info
  examples: |
    - Do you have a [website](information)
    - can i have your [web address](information)
    - Do you maintain a [website](information)
    - [webpage](information) please
    - tell me more about your [site](information)
    - what about your [website](information)
    - I want to know about your [website](information)
    - do you have a [website](information)
    - do you have a [website](information)
- intent: inform
  examples: |
    - Hi I am [steve](name)
    - Hello this is [john](name)
    - my name is [rakesh](name)
    - this is [aleksey](name)
    - you can call me [riya](name)
    - i am [rashmi](name)
    - my name is [stella](name)
    - this is [gayatri](name)
    - [angela](name)
    - Hi I am [minakshi](name)
    - hello [rachel](name) here
    - [rakesh](name)
    - ciao you can call me [sam](name)
    - I am [minakshi](name)
    - you can call me [vincent](name)
    - i am [minakshi](name)

"""

%store nlu_yml >./data/nlu.yml

Writing 'nlu_yml' (str) to file './data/nlu.yml'.


### Rules & Stories
Replace rules.yml & stories.yml  
rules.yml file contains training data used to train your assistant's dialogue management model. Rules describe short pieces of conversations that should always follow the same path. Stories can be used to train models that are able to generalize to unseen conversation paths.  
Read more at the following links: 
- https://rasa.com/docs/rasa/rules
- https://rasa.com/docs/rasa/stories

In [3]:
rules_yml = """
version: "3.1"

rules:

- rule: Say hi anytime the user says hi
  steps:
  - intent: greet
  - action: utter_greet

- rule: Say goodbye anytime the user says goodbye
  steps:
  - intent: goodbye
  - action: utter_goodbye

- rule: Say 'I am a bot' anytime the user challenges
  steps:
  - intent: bot_challenge
  - action: utter_iamabot

- rule: answer q1
  steps:
  - intent: q1
  - action: utter_a1

- rule: answer q1
  steps:
  - intent: q1
  - action: utter_a1
  
- rule: answer q2
  steps:
  - intent: q2
  - action: utter_a2
  
- rule: answer q3
  steps:
  - intent: q3
  - action: utter_a3
 
- rule: answer q4
  steps:
  - intent: q4
  - action: utter_a4
 
- rule: answer q5
  steps:
  - intent: q5
  - action: utter_a5
 
- rule: answer q6
  steps:
  - intent: q6
  - action: utter_a6
 
- rule: answer q7
  steps:
  - intent: q7
  - action: utter_a7
 
- rule: answer q8
  steps:
  - intent: q8
  - action: utter_a8
  
- rule: answer q9
  steps:
  - intent: q9
  - action: utter_a9
  
- rule: answer q10
  steps:
  - intent: q10
  - action: utter_a10
 
- rule: answer q11
  steps:
  - intent: q11
  - action: utter_a11
 
- rule: answer q12
  steps:
  - intent: q12
  - action: utter_a12
  
- rule: answer q13
  steps:
  - intent: q13
  - action: utter_a13
 
- rule: answer q14
  steps:
  - intent: q14
  - action: utter_a14
 
- rule: answer q15
  steps:
  - intent: q15
  - action: utter_a15
  
- rule: answer q16
  steps:
  - intent: q16
  - action: utter_a16
 
- rule: answer q17
  steps:
  - intent: q17
  - action: utter_a17
 
- rule: answer q18
  steps:
  - intent: q18
  - action: utter_a18
 
- rule: answer q19
  steps:
  - intent: q19
  - action: utter_a19
 
- rule: answer q20
  steps:
  - intent: q20
  - action: utter_a20
 
- rule: answer q21
  steps:
  - intent: q21
  - action: utter_a21

- rule: IF none of the above questions GOTO google snippet
  steps:
  - intent: nlu_fallback
  - action: action_google

"""

%store rules_yml >./data/rules.yml

stories_yml = """
version: "3.1"

stories:

"""
%store stories_yml >./data/stories.yml

Writing 'rules_yml' (str) to file './data/rules.yml'.
Writing 'stories_yml' (str) to file './data/stories.yml'.


### NLU Model Tuning 
Replace config.yml  
config.yml file contains the main pipeline and policies.  
Read more at the following links:
 - https://rasa.com/docs/rasa/tuning-your-model/#how-to-choose-a-pipeline
 - https://rasa.com/docs/rasa/policies/

In [4]:
config_yml = """
language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: DIETClassifier
    epochs: 100
    constrain_similarities: true
  - name: EntitySynonymMapper
  - name: ResponseSelector
    epochs: 100
    constrain_similarities: true
  - name: FallbackClassifier
    threshold: 0.8
    
policies:
  - name: MemoizationPolicy
  - name: RulePolicy
    core_fallback_threshold: 0.4
    core_fallback_action_name: "action_default_fallback"
    enable_fallback_prediction: True

"""

%store config_yml >./config.yml

Writing 'config_yml' (str) to file './config.yml'.


### Domain
Replace domain.yml  
domain.yml file defines the universe in which your assistant operates. It specifies the intents, entities, slots, responses, forms, and actions your bot should know about. It also defines a configuration for conversation sessions.  
Read more at the following links: 
 - https://rasa.com/docs/rasa/domain
 - https://rasa.com/docs/rasa/responses
 - https://rasa.com/docs/rasa/training-data-format#entities

In [5]:
domain_yml = """
version: '3.1'
intents:
- more info:
    is_retrieval_intent: true
- affirm
- inform
- goodbye
- greet
- deny
- general
- stop
- browse
- thanks
- reference
- acknowledge_info
- unhappy
- bot_challenge
- q1
- q2
- q3
- q4
- q5
- q6
- q7
- q8
- q9
- q10
- q11
- q12
- q13
- q14
- q15
- q16
- q17
- q18
- q19
- q20
- q21
responses:
  utter_greet:
  - text: Hey! Ask me anything you want about Nutrition, Plant Based Diets etc.
  - text: Hello! I am here to help you. Ask me anything you want about Nutrition, Plant Based Diets etc.
  utter_cheer_up:
  - text: 'Here is something to cheer you up:'
    image: https://i.imgur.com/nGF1K8f.jpg
  utter_did_that_help:
  - text: Did that help you?
  utter_happy:
  - text: Great, carry on!
  utter_goodbye:
  - text: Bye
  - text: See you next time...
  utter_iamabot:
  - text: I am a bot, powered by Rasa. Ask me anything you want about Nutrition, Diet etc.
  utter_info:
  - text: here is the link to our [website](https://www.fellowship.ai/)
  utter_acknowledge_info:
  - text: Thank you!
  utter_general:
  - text: I am here if you need me...
  utter_unhappy:
  - text: Ohh... I am sorry!!!
  - text: Could you describe what you have in mind?
  - text: Sorry! Please try again...
  utter_affirm:
  - text: Anything else I can help you with...
  utter_please_rephrase:
  - text: I'm sorry, I didn't quite understand that. Could you rephrase?
  utter_a1:
  - text: ornish and esselstyn proved that even advanced crippling heart disease could not only just be prevented and treated, but reversed, with a plant-based diet, centered around grains, beans, vegetables, fruits, with nuts and seeds treated as condiments—and no oils, dairy, meat, poultry, or fish.
  utter_a2:
  - text: Our results on miso-soup have shown that anti-hypertensive effect of miso is possiblt above hypertensive effect of salt.
  utter_a3:
  - text: For the breast cancer survivor, current evidence suggests no adverse effects on reccurrence of survival from consuming soy and soy foods, and there is the potential for these foods to exert a positive synergistic effect with tamoxifen.
  utter_a4:
  - text: In this meta-analysis there was a 11-24% reduction in the relative risk of coronary heart disease, stroke, cardiovascular disease, total cancer, and all-cause mortality with a higher nut intake, respectively. In the dose-response analysis there was a 7-29% resuction in the relative risk for a one serving per day increase in nut intake (one serving = 28 grams), respectively.
  utter_a5:
  - text: Cloves, ginger, rosemary, and turmeric.
  utter_a6:
  - text: keto is not a plant-based diet. it is a low-carb, low-fat, high-protein diet. there is no difference between a keto diet and a vegan diet.
  utter_a7:
  - text: no, it is not safe to get pregnant on a plant based diet. there are a number of reasons for this, but the most important one is that you need to be getting enough b12. if you don't take your b12, you will develop preeclampsia, pre-gravid obesity, and other health problems.
  utter_a8:
  - text: no, they are not more prone to developing alzheimer's. they are more likely to develop alzheimer's if they have a genetic predisposition to the disease.
  utter_a9:
  - text: type 1 diabetes is caused by a lack of insulin, which is a hormone that is produced by the pancreas. there are two types of diabetes. type 1 and type 2. type 1 is a type of diabetes caused by an excess of insulin in the body. type 2 is caused when the body doesn't produce enough insulin to compensate for the excess sugar in the blood. plant-based diets have been shown to lower the risk of type 1 diabetes.
  utter_a10:
  - text: there are a number of plants that have been shown to be anti-inflammatory. some of the most well-known are potatoes, broccoli, cauliflower, and kale.
  utter_a11:
  - text: plant-based diets do not prevent heart disease, but they do reduce the risk of heart disease.
  utter_a12:
  - text: a bmi of 18.5 to 24.9 is considered “ideal weight.” a bmi over 30 is considered obese.
  utter_a13:
  - text: i'm not a nutritionist, but i do have a background in food science, so i'll try to answer your question as best as i can. the short answer to your question is yes, green smoothies are good for you. the long answer is no, they are bad for you, because they contain a lot of sugar, which is bad for your body.
  utter_a14:
  - text: no, there are no foods with negative calories. there are, however, foods that take more energy to digest than they provide.
  utter_a15:
  - text: almond milk contains a lot of protein, which is good for you. however, it also contains a ton of sugar, which can be bad for you, especially if you're trying to lose weight.
  utter_a16:
  - text: no, you don't need to add salt while eating vegetarian. vegetarians get about 3,500 mg of sodium a day, which is about the equivalent of about a teaspoon and a half of table salt.
  utter_a17:
  - text: a plant-based diet is a diet that is mostly made up of vegetables, fruits, whole grains, nuts, seeds, and spices.
  utter_a18:
  - text: plant-based diets can help you lose weight, it can help prevent and treat diabetes and heart disease, lower blood pressure, and can help your live longer.
  utter_a19:
  - text: it depends on what you mean by "how much protein do you really need". there is an average minimum protein requirement for humans, and in the us it is 0.75 to 1.0 grams per kilogram of body weight daily, so if you weigh 220 pounds, or 100 kilograms, that is about 80 to 100 grams of protein.
  utter_a20:
  - text: vitamin b12 is one of the most important vitamins in the human body, and it's found in a lot of plant-based foods. if you don't have b12 in your diet, you're going to need to take vitamin b12 supplements.
  utter_a21:
  - text: prebiotics are what your good bacteria eat. probiotics are the good bacteria themselves. prebiotics provide indigestible food components that stimulate the growth and activity of beneficial bacteria, usually by serving as their food supply.
  utter_default:
  - text: Sorry I didn't get that. Can you rephrase?
session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true
entities:
- illness
- ingredient
- substance
- user_age
- user_gender
- user_given_reference
- user_mood
- information
- intention
- name
- cure
- food
- pregnancy
actions:
- utter_goodbye
- action_get_advice
- action_recieve_name
- action_utter_purchase
- utter_acknowledge_info
- utter_affirm
- utter_did_that_help
- utter_general
- utter_greet
- utter_info
- action_google

"""

%store domain_yml >./domain.yml

Writing 'domain_yml' (str) to file './domain.yml'.


### Custom Actions
Replace actions.py  
actions.py file 
Read more at the following links: 
 - https://rasa.com/docs/rasa/actions/
 - https://rasa.com/docs/rasa/custom-actions
 - https://rasa.com/docs/rasa/default-actions

In [6]:
actions_py = """
#GOOGLE FEATURED SNIPPET DEPENDENCIES
import requests
import bs4
import re


#DEFAULT DEPENDENCIES
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

class ActionGoogle(Action):

    def name(self) -> Text:
        return "action_google"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        query = tracker.latest_message['text']
        if query != "":
            url = 'https://google.com/search?q=' + query 
            http_headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36'}    
            # response = requests.get(url, headers=http_headers)
            response = requests.get(url, headers=http_headers)
            soup = bs4.BeautifulSoup(response.text,"html.parser")
            snipped_title = soup.find(string=re.compile("snippet"))
            if snipped_title:
                feat_snip = snipped_title.parent.parent
                # print(feat_snip)
                feat_snip_text = feat_snip.get_text(' ', strip=True)
                #TEXT SANITIZE
                if "http" in feat_snip_text:
                    pos = feat_snip_text.find("http")
                    feat_snip_text = feat_snip_text[:pos]
                months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                for month in months:
                    if month in feat_snip_text:
                        pos = feat_snip_text.find(month)
                        feat_snip_text = feat_snip_text[:pos]
                dispatcher.utter_message(text=feat_snip_text)
            else:
                dispatcher.utter_message(response="utter_default")
        return []

"""

%store actions_py >./actions/actions.py

endpoints_yml = """
action_endpoint:
 url: "http://localhost:5055/webhook"
 
"""

%store endpoints_yml >./endpoints.yml

Writing 'actions_py' (str) to file './actions/actions.py'.
Writing 'endpoints_yml' (str) to file './endpoints.yml'.


### Credentials
Replace credentials.yml  
credentials.yml file contains the connectors that RASA can use.  
Read more at the following links: 
 - https://rasa.com/docs/rasa/messaging-and-voice-channels/
 - https://rasa.com/docs/rasa/connectors/telegram/#getting-credentials

In [7]:
credentials_yml = f"""
rest:

rasa:
  url: "http://localhost:5002/api"

"""

%store credentials_yml >./credentials.yml

Writing 'credentials_yml' (str) to file './credentials.yml'.


## Runtime

### spaCy validation table

In [8]:
!spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.2.4) =================
ℹ spaCy installation: /usr/local/lib/python3.9/dist-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_lg   >=3.2.0,<3.3.0   3.2.0   ✔



### Train a new RASA model

In [9]:
!rasa train
import os

model_path = f"./models/{sorted(os.listdir('./models'))[-1]}"
print(model_path)

/usr/local/lib/python3.9/dist-packages/rasa/engine/recipes/recipe.py:35: FutureWarning: From Rasa Open Source 4.0.0 onwards it will be required to specify a recipe in your model configuration. Defaulting to recipe 'default.v1'.
  rasa.shared.utils.io.raise_deprecation_warning(
/usr/local/lib/python3.9/dist-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/dist-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:47: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_version = LooseVersion(INCLUSIVE_MIN_TF_VERSION)
2022-09-19 11:01:04 INFO     rasa.engine.training.hooks  - Restored component 

### Test the newly created model

In [10]:
!rasa test

/usr/local/lib/python3.9/dist-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/dist-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:47: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_version = LooseVersion(INCLUSIVE_MIN_TF_VERSION)
2022-09-19 11:01:12 INFO     rasa.core.processor  - Loading model models/20220919-110059-scared-schema.tar.gz...
2022-09-19 11:01:35 INFO     rasa.shared.utils.validation  - The 'version' key is missing in the training data file /notebooks/tests/test_stories.yml. Rasa Open Source will read the file as a version '3.1' file. See https://rasa.com/docs/rasa/

### Run the bot

In [11]:
!rasa run actions

/usr/local/lib/python3.9/dist-packages/sanic_cors/extension.py:39: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  SANIC_VERSION = LooseVersion(sanic_version)
2022-09-19 11:02:11 INFO     rasa_sdk.endpoint  - Starting action endpoint server...
2022-09-19 11:02:12 INFO     rasa_sdk.executor  - Registered function for 'action_google'.
2022-09-19 11:02:12 INFO     rasa_sdk.endpoint  - Action endpoint is up and running on http://0.0.0.0:5055
^C
